Creating a new database

In [ ]:
IF NOT EXISTS (SELECT * FROM master.dbo.sysdatabases WHERE name = 'externalDB')
    BEGIN
        CREATE DATABASE externalDB
END;
SELECT * FROM sys.databases;
GO
USE externalDB;
GO

Create a master key in the database

In [ ]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Password1234';

Create a scoped credential for MongoDB (with login credentials)

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL MongoDB
WITH IDENTITY = 'root', SECRET = 'password123'

Create external data source using MongoDB login credentials

In [ ]:
CREATE EXTERNAL DATA SOURCE MongoDataSource
WITH (
    LOCATION = 'mongodb://10.10.2.218:27017',
    CREDENTIAL = MongoDB,
    connection_options = 'UseDefaultEncryptionOptions=false'
);

(FOR DEBUGGING)

Verify that scoped scoped credentials and data sources were created

In [ ]:
USE externalDB
SELECT * FROM sys.database_scoped_credentials
SELECT * FROM sys.external_data_sources

Create external table for MongoDB

In [ ]:
BEGIN TRY
    BEGIN TRANSACTION T3cb6d4aaec3e45e89d78784f8a44817
        USE [externalDB];
        EXEC(N'CREATE SCHEMA [airportData]');
        CREATE EXTERNAL TABLE [airportData].[mailVolume]
        (
            [_id] NVARCHAR(24) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL,
            [month] NVARCHAR(4000) COLLATE SQL_Latin1_General_CP1_CI_AS,
            [Total Mail] NVARCHAR(4000) COLLATE SQL_Latin1_General_CP1_CI_AS
        )
        WITH (LOCATION = N'[airportData].[mailVolume]', DATA_SOURCE = [MongoDataSource]);
    COMMIT TRANSACTION T3cb6d4aaec3e45e89d78784f8a44817
END TRY
BEGIN CATCH
    IF @@TRANCOUNT > 0
        ROLLBACK TRANSACTION T3cb6d4aaec3e45e89d78784f8a44817
    DECLARE @ErrorMessage NVARCHAR(max) = ERROR_MESSAGE();
    DECLARE @ErrorSeverity INT = ERROR_SEVERITY();
    DECLARE @ErrorState INT = ERROR_STATE();
    RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);
END CATCH;


Data querying

In [ ]:
SELECT TOP (1000) [_id]
      ,[month]
      ,[Total Mail]
  FROM [externalDB].[airportData].[mailVolume]

Joining data from the two tables

In [ ]:
SELECT [airportdata].[mailVolume].[month], [airportdata].[mailVolume].[Total Mail],
[dbo].[MonthlyArrivals].[Total_Aircraft_Arrivals_And_Departures], [dbo].[MonthlyArrivals].[Total_Passengers]
FROM [airportdata].[mailVolume]
JOIN [dbo].[MonthlyArrivals]
ON [airportdata].[mailVolume].[month] = [dbo].[MonthlyArrivals].[month]

Dropping the tables

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[airportData].[mailVolume]') AND type in (N'U'))
DROP EXTERNAL TABLE [airportData].[mailVolume]
GO
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[MonthlyArrivals]') AND type in (N'U'))
DROP EXTERNAL TABLE [dbo].[MonthlyArrivals]
GO

Dropping the database

In [ ]:
USE master;
GO
ALTER DATABASE externalDB SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
GO
DROP DATABASE externalDB;
GO